In [51]:
import os

folders = [
    'random',
    'padding',
    'symmetric',
]
functions = [
    'random',
    'pad',
    'encrypt',
]
category_names = [
    'Random Number Generators',
    'Block Cipher Padding',
    'Symmetric Encryption',
]
python_suffix = '.py'

files = {}
file_count = 0
for folder in folders:
    files[folder] = {}
    for file_name in os.listdir(folder):
        with open(os.path.join(folder, file_name), 'r') as file:
            name = file_name.removesuffix(python_suffix)
            files[folder][name] = file.read()
            file_count += 1
        
file_count

16

In [52]:
from radon.complexity import cc_rank, cc_visit
from radon.metrics import h_visit

def get_function_statistics(code: str, function_name: str) -> dict | None:
    # Run analyzers
    cc = cc_visit(code) # Cyclomatic complexity
    h = h_visit(code) # Halstead metrics
    
    # Find CC function
    cc_function = list(filter(lambda x: x.name == function_name, cc))
    if len(cc_function) != 1:
        return None
    cc_function = cc_function[0]
    
    # Find Halstead function
    h_function = list(filter(lambda x: x[0] == function_name, h.functions))
    if len(h_function) != 1:
        return None
    h_function = h_function[0][1] # Get the function's metrics object
    
    # Collect statistics
    stats = {
        'lines': cc_function.endline - cc_function.lineno,
        'cc': cc_function.complexity,
        'cc_rank': cc_rank(cc_function.complexity),
        'h_volume': h_function.volume,
        'h_length': h_function.length,
        'h_difficulty': h_function.difficulty,
    }
    return stats

In [53]:
statistics = {}
for folder, function in zip(folders, functions):
    statistics[ folder ] = {}
    for name, code in files[folder].items():
        statistics[ folder ][ name ] = get_function_statistics(code, function)

list(statistics[ list(statistics.keys())[0] ].items())[0]

('xorshift',
 {'lines': 6,
  'cc': 1,
  'cc_rank': 'A',
  'h_volume': 120.0,
  'h_length': 30,
  'h_difficulty': 4.545454545454546})

In [58]:
def markdown_table_from_statistics(statistics):
    global symmetrics_folter, python_suffix
    table = '| Name | Lines | CC | CC Rank | Halstead Volume | Halstead Length | Halstead Difficulty |\n'
    table += '|-|-|-|-|-|-|-|\n'
    for name, stats in statistics.items():
        table += f'| [{name}]({symmetrics_folter}/{name}{python_suffix}) | {stats["lines"]} | {stats["cc"]} | {stats["cc_rank"]} | {stats["h_volume"]:.1f} | {stats["h_length"]} | {stats["h_difficulty"]:.1f} |\n'
    return table

def make_all_tables(folders, category_names, statistics):
    text = '## Implementations\n\n'
    for folder, category_name in zip(folders, category_names):
        text += f'### {category_name}\n'
        text += markdown_table_from_statistics(statistics[folder])
        text += '\n'
    return text
        
with open('README.temp.md', 'w') as file:
    file.write(make_all_tables(folders, category_names, statistics))